# 0. Imports

In [11]:
import pandas as pd
import dill
import csv
import requests
import datetime
import holidays
import pygame
import time
import os


from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split, cross_val_score
from xgboost import XGBClassifier


# 1. Loading Data, Type Casting, Binarizing Target, and Merging Datasets

## 1.1 Loading Data

## 1.1.1 Hits

In [12]:
ga_hits = os.path.join('..', 'data/csv/ga_hits.csv')

df_hits = pd.read_csv(
    ga_hits,
    dtype={
        "hit_time": "float64", 
        "hit_number":"Int32"
        }
)

df_hits.shape

(15726470, 11)

In [13]:
df_hits = df_hits[["session_id", "event_action"]]

In [14]:
df_hits.head()

,session_id,event_action
0,5639623078712724064.1640254056.1640254056,quiz_show
1,7750352294969115059.1640271109.1640271109,quiz_show
2,885342191847998240.1640235807.1640235807,quiz_show
3,142526202120934167.1640211014.1640211014,quiz_show
4,3450086108837475701.1640265078.1640265078,quiz_show


In [15]:
df_hits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15726470 entries, 0 to 15726469
Data columns (total 2 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   session_id    object
 1   event_action  object
dtypes: object(2)
memory usage: 240.0+ MB


## 1.1.2 Sessions

In [16]:
ga_sessions = os.path.join('..', 'data/csv/ga_sessions.csv')
df_sessions = pd.read_csv(
    ga_sessions,
    dtype={"visit_number":"Int32"},
    low_memory=False
)

df_sessions.shape

(1860042, 18)

In [17]:
df_sessions.head()

,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_model,device_screen_resolution,device_browser,geo_country,geo_city
0,9055434745589932991.1637753792.1637753792,2108382700.1637753791,2021-11-24,14:36:32,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,NaN,360x720,Chrome,Russia,Zlatoust
1,905544597018549464.1636867290.1636867290,210838531.1636867288,2021-11-14,08:21:30,1,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,xhoenQgDQsgfEPYNPwKO,IGUCNvHlhfHpROGclCit,mobile,Android,Samsung,NaN,385x854,Samsung Internet,Russia,Moscow
2,9055446045651783499.1640648526.1640648526,2108385331.1640648523,2021-12-28,02:42:06,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,NaN,360x720,Chrome,Russia,Krasnoyarsk
3,9055447046360770272.1622255328.1622255328,2108385564.1622255328,2021-05-29,05:00:00,1,kjsLglQLzykiRbcDiGcD,cpc,NaN,NOBKLgtuvqYWkXQHeYWM,NaN,mobile,NaN,Xiaomi,NaN,393x786,Chrome,Russia,Moscow
4,9055447046360770272.1622255345.1622255345,2108385564.1622255328,2021-05-29,05:00:00,2,kjsLglQLzykiRbcDiGcD,cpc,NaN,NaN,NaN,mobile,NaN,Xiaomi,NaN,393x786,Chrome,Russia,Moscow


In [18]:
def map_dtype(dtype):
    if pd.api.types.is_integer_dtype(dtype):
        return int
    elif pd.api.types.is_float_dtype(dtype):
        return float
    else:
        return str

column_types = {col: map_dtype(dtype) for col, dtype in df_sessions.dtypes.items()}
column_types

{'session_id': str,
 'client_id': str,
 'visit_date': str,
 'visit_time': str,
 'visit_number': int,
 'utm_source': str,
 'utm_medium': str,
 'utm_campaign': str,
 'utm_adcontent': str,
 'utm_keyword': str,
 'device_category': str,
 'device_os': str,
 'device_brand': str,
 'device_model': str,
 'device_screen_resolution': str,
 'device_browser': str,
 'geo_country': str,
 'geo_city': str}

## 1.2 Binarizing Target

In [19]:
action_list = [
    'sub_car_claim_click', 
    'sub_car_claim_submit_click', 
    'sub_open_dialog_click', 
    'sub_custom_question_submit_click', 
    'sub_call_number_click', 
    'sub_callback_submit_click', 
    'sub_submit_success', 
    'sub_car_request_submit_click'
    ]

df_hits.event_action = df_hits.event_action.apply(lambda x: 1 if x in action_list else 0)
df_hits.event_action.value_counts()

event_action
0    15621562
1      104908
Name: count, dtype: int64

## 1.3 Merge

### 1.3.1 Merging

In [20]:
df = pd.merge(left=df_sessions, right=df_hits, on='session_id', how='inner')
df.shape

(15685219, 19)

In [21]:
df.head()

,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_model,device_screen_resolution,device_browser,geo_country,geo_city,event_action
0,9055434745589932991.1637753792.1637753792,2108382700.1637753791,2021-11-24,14:36:32,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,NaN,360x720,Chrome,Russia,Zlatoust,0
1,9055434745589932991.1637753792.1637753792,2108382700.1637753791,2021-11-24,14:36:32,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,NaN,360x720,Chrome,Russia,Zlatoust,0
2,905544597018549464.1636867290.1636867290,210838531.1636867288,2021-11-14,08:21:30,1,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,xhoenQgDQsgfEPYNPwKO,IGUCNvHlhfHpROGclCit,mobile,Android,Samsung,NaN,385x854,Samsung Internet,Russia,Moscow,0
3,9055446045651783499.1640648526.1640648526,2108385331.1640648523,2021-12-28,02:42:06,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,NaN,360x720,Chrome,Russia,Krasnoyarsk,0
4,9055446045651783499.1640648526.1640648526,2108385331.1640648523,2021-12-28,02:42:06,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,NaN,360x720,Chrome,Russia,Krasnoyarsk,0


### 1.3.2 Dropping duplicates

In [22]:
df = df.sort_values(by=["event_action"], ascending=False)
df = df.drop_duplicates(subset=["session_id"], keep="first")
df.shape

(1732266, 19)

# 2. Data Preparation

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1732266 entries, 12337081 to 5245208
Data columns (total 19 columns):
 #   Column                    Dtype 
---  ------                    ----- 
 0   session_id                object
 1   client_id                 object
 2   visit_date                object
 3   visit_time                object
 4   visit_number              Int32 
 5   utm_source                object
 6   utm_medium                object
 7   utm_campaign              object
 8   utm_adcontent             object
 9   utm_keyword               object
 10  device_category           object
 11  device_os                 object
 12  device_brand              object
 13  device_model              object
 14  device_screen_resolution  object
 15  device_browser            object
 16  geo_country               object
 17  geo_city                  object
 18  event_action              int64 
dtypes: Int32(1), int64(1), object(17)
memory usage: 259.4+ MB


In [30]:
df_data_prep = df.copy()

## 2.1 Pandas Profiling

In [31]:
# report = ProfileReport(df_data_prep)
# report

## 2.2 Processing NANs

Let's have a closer look at columns with NAN values.

In [32]:
def get_na_dict(df):
    na_dict = dict()
    for i in df.columns:
        na_value = df[i].isna().sum()
        if na_value > 0:
            percent = round(na_value / df.shape[0] * 100, 2)
            na_dict[i] = na_value, percent
    
    return na_dict if na_dict else "No NAN values in the dataframe"


def sort_dict(dct):
    sorted_list = sorted(dct.items(), key=lambda x: x[1][0], reverse=True)

    return sorted_list


def print_list(sorted_list):
    for i in sorted_list:
        print(f"{i[0]:<25}: {i[1][0]:<10} = {i[1][1]} %")


def get_na(df):
    na_dict = get_na_dict(df_data_prep)
    sorted_list = sort_dict(na_dict)
    print_list(sorted_list)
    return sorted_list


get_na(df_data_prep)


device_model             : 1717204    = 99.13 %
utm_keyword              : 1020752    = 58.93 %
device_os                : 1013964    = 58.53 %
device_brand             : 347196     = 20.04 %
utm_adcontent            : 304137     = 17.56 %
utm_campaign             : 195287     = 11.27 %
utm_source               : 76         = 0.0 %


[('device_model', (1717204, 99.13)),
 ('utm_keyword', (1020752, 58.93)),
 ('device_os', (1013964, 58.53)),
 ('device_brand', (347196, 20.04)),
 ('utm_adcontent', (304137, 17.56)),
 ('utm_campaign', (195287, 11.27)),
 ('utm_source', (76, 0.0))]

### 2.2.1 Filling device OS

In [33]:
df_data_prep.device_os.value_counts(dropna=False)

device_os
NaN              1013964
Android           425520
iOS               182597
Windows            81740
Macintosh          23415
Linux               4616
(not set)            309
Chrome OS             65
BlackBerry            24
Tizen                  7
Firefox OS             3
Windows Phone          2
Nokia                  2
Samsung                2
Name: count, dtype: int64

In [34]:
dev_filter = df_data_prep[
    (df_data_prep['device_brand'] == 'Apple') & \
    (df_data_prep['device_os'].isna() == True) 
]
dev_filter.head()

,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_model,device_screen_resolution,device_browser,geo_country,geo_city,event_action
4773100,3277696661413753280.1628238272.1628238272,763148223.1628238272,2021-08-06,11:00:00,1,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,NaN,mobile,NaN,Apple,NaN,414x896,Chrome,Russia,Grozny,1
9390443,5723681030410214065.1631460017.1631460017,1332648338.1631460017,2021-09-12,18:00:00,1,QxAxdyPLuQMEcrdZWdWb,cpm,EiQppLFrUZrUsjXVulLg,JNHcPlZPxEMWDnRiyoBf,NaN,mobile,NaN,Apple,NaN,414x896,Safari (in-app),Russia,Saint Petersburg,1
1770382,1695172405933607602.1622641330.1622641330,394688082.1622641330,2021-06-02,16:00:00,1,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,NaN,mobile,NaN,Apple,NaN,414x896,Safari,Russia,Moscow,1
13830772,8072712495578074887.1623838471.1623838471,1879574846.1623838471,2021-06-16,13:00:00,1,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,NaN,mobile,NaN,Apple,NaN,414x896,Safari (in-app),Russia,Saint Petersburg,1
15482720,8947424787755135500.1622056717.1622056717,2083234672.1621848588,2021-05-26,22:00:00,2,kjsLglQLzykiRbcDiGcD,organic,LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,NaN,mobile,NaN,Apple,NaN,390x844,Safari,Russia,Samara,1


In [35]:
def fill_device_os(row):
    if pd.isna(row.device_os):
        if row.device_brand == 'Apple':
            return 'iOS'
        elif row.device_category == 'mobile':
            return 'Android'
        elif row.device_category == 'desktop':
            return 'Windows'
        else:
            return 'other'
    else:
        return row['device_os']


df_data_prep['device_os'] = df_data_prep.apply(fill_device_os, axis=1)

In [36]:
df_data_prep.device_os.value_counts(dropna=False)

device_os
Android          875038
iOS              503538
Windows          318491
Macintosh         23415
other              6754
Linux              4616
(not set)           309
Chrome OS            65
BlackBerry           24
Tizen                 7
Firefox OS            3
Windows Phone         2
Nokia                 2
Samsung               2
Name: count, dtype: int64

In [37]:
df_data_prep = df_data_prep[~(df_data_prep.device_os == '(not set)')]
diff = df.shape[0] - df_data_prep.shape[0]
diff


309

### 2.2.2 Filling device brand

In [38]:
get_na(df_data_prep)

device_model             : 1716902    = 99.13 %
utm_keyword              : 1020739    = 58.94 %
device_brand             : 346894     = 20.03 %
utm_adcontent            : 304123     = 17.56 %
utm_campaign             : 195282     = 11.28 %
utm_source               : 76         = 0.0 %


[('device_model', (1716902, 99.13)),
 ('utm_keyword', (1020739, 58.94)),
 ('device_brand', (346894, 20.03)),
 ('utm_adcontent', (304123, 17.56)),
 ('utm_campaign', (195282, 11.28)),
 ('utm_source', (76, 0.0))]

In [39]:
df_data_prep.device_brand.value_counts()

device_brand
Apple        503533
Samsung      311641
Xiaomi       269251
Huawei       173828
Realme        17926
              ...  
Maze              1
Dell              1
Gigaset           1
Centric           1
Honeywell         1
Name: count, Length: 200, dtype: int64

In [40]:
df_data_prep[['device_category', 'device_brand']].groupby('device_category').agg(
    missing_brands=('device_brand', lambda x: x.isna().sum())
    )

,missing_brands
device_category,
desktop,346450
mobile,288
tablet,156


In [41]:
df_data_prep[
    (df_data_prep["device_brand"].isna() == True) &
    (df_data_prep["device_os"] == "Macintosh")
].shape

(23415, 19)

In [42]:
df_data_prep[
    (df_data_prep["device_brand"].isna() == True) &
    (df_data_prep["device_os"] == "Windows")
].shape

(318444, 19)

In [43]:
df_data_prep[
    (df_data_prep["device_brand"].isna() == True) &
    (df_data_prep["device_os"] == "Linux")
].shape

(4616, 19)

In [44]:
def fill_device_brand(row):
    if pd.isna(row.device_brand):
        if row.device_os == 'Macintosh':
            return 'Apple'
        elif row.device_os == "Windows" or row.device_os == "Linux":
            return 'custom'
        else:
            return 'other'
    else:
        return row['device_brand']


df_data_prep['device_brand'] = df_data_prep.apply(fill_device_brand, axis=1)

In [45]:
get_na(df_data_prep)

device_model             : 1716902    = 99.13 %
utm_keyword              : 1020739    = 58.94 %
utm_adcontent            : 304123     = 17.56 %
utm_campaign             : 195282     = 11.28 %
utm_source               : 76         = 0.0 %


[('device_model', (1716902, 99.13)),
 ('utm_keyword', (1020739, 58.94)),
 ('utm_adcontent', (304123, 17.56)),
 ('utm_campaign', (195282, 11.28)),
 ('utm_source', (76, 0.0))]

Now, let's prepare two lists: one—with columns to drop (threshold > 30%), and the other—with columns where NANs to be filled with a mode.

### 2.2.3 Deleting Cols With Lots of NANs

In [46]:
def cols_to_drop(sorted_list):
    columns_to_drop = []
    columns_to_process = []
    for i in sorted_list:
        if i[1][1] > 30:
            columns_to_drop.append(i[0])
        else:
            columns_to_process.append(i[0])
    
    return columns_to_drop, columns_to_process



columns_to_drop, columns_to_process = cols_to_drop(get_na(df_data_prep))
print("\n\nThe following columns will be dropped:\n")
print(*columns_to_drop, sep="\n")


device_model             : 1716902    = 99.13 %
utm_keyword              : 1020739    = 58.94 %
utm_adcontent            : 304123     = 17.56 %
utm_campaign             : 195282     = 11.28 %
utm_source               : 76         = 0.0 %


The following columns will be dropped:

device_model
utm_keyword


Dropping the columns:

In [47]:
df_data_prep = df_data_prep.drop(columns_to_drop, axis=1)
df_data_prep.shape

(1731957, 17)

Let's also drop id's and hit_type columns. The latter should be eliminated because it has only one value according to Ydata Profiling.

In [48]:
# ids = ["session_id", "client_id", "hit_type"]
# df_data_prep = df_data_prep.drop(ids, axis=1)
# df_data_prep.shape

### 2.2.2 Filling NANs with a Mode

Now let's fill all the NANs with a mode.

In [49]:
isna = df_data_prep.isna().sum()
isna

session_id                       0
client_id                        0
visit_date                       0
visit_time                       0
visit_number                     0
utm_source                      76
utm_medium                       0
utm_campaign                195282
utm_adcontent               304123
device_category                  0
device_os                        0
device_brand                     0
device_screen_resolution         0
device_browser                   0
geo_country                      0
geo_city                         0
event_action                     0
dtype: int64

In [50]:
def fill_with_mode(df):
    for i in df.columns:
        df[i] = df[i].fillna(df[i].mode()[0])

    return df

df_data_prep = fill_with_mode(df_data_prep)


In [51]:
df_ones = df_data_prep[~(df_data_prep['event_action'] == 0)]
df_ones.head()

,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,device_category,device_os,device_brand,device_screen_resolution,device_browser,geo_country,geo_city,event_action
12337081,7284725054902084534.1624538941.1624538941,1696107223.1607705526,2021-06-24,15:00:00,75,jaSOmLICuBzCFqHfBdRg,email,LliRUcMuIXWdLyWHGyiO,sDWYAbLNiGZVxGBDdTxc,desktop,Windows,custom,1280x720,Chrome,Russia,Moscow,1
3288869,2502982653972912560.1628266799.1628266799,582771062.1627724208,2021-08-06,19:00:00,6,kjsLglQLzykiRbcDiGcD,organic,LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,desktop,Windows,custom,1536x864,Firefox,Russia,Moscow,1
10566794,63501960694916780.1640162989.1640162989,14785202.1640162988,2021-12-22,11:49:49,1,ZpYIoDJMcFzVoPFsHGJL,push,sbJRYgVfvcnqKJNDDYIr,JNHcPlZPxEMWDnRiyoBf,mobile,Android,Samsung,412x915,Chrome,Russia,Moscow,1
14538018,8447975741139857132.1640115950.1640115969,1966947629.1640115948,2021-12-21,22:46:09,1,bByPQxmDaMXgpHeypKSM,referral,QdLfySaGXolfTBSNVfHn,SOkCdPxfUcZUzzOdgGES,desktop,Windows,custom,1111x625,Chrome,Russia,Kazan,1
12116165,7169040235355732641.1625062049.1625062049,1669172252.1625062049,2021-06-30,17:00:00,1,ZpYIoDJMcFzVoPFsHGJL,banner,eimRuUrNhZLAYcwRrNXu,JNHcPlZPxEMWDnRiyoBf,mobile,Android,Samsung,339x716,Chrome,Russia,Saint Petersburg,1


In [52]:
df_zeros = df_data_prep[~(df_data_prep['event_action'] == 1)]
df_zeros.head()

,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,device_category,device_os,device_brand,device_screen_resolution,device_browser,geo_country,geo_city,event_action
10462439,629540588298749367.1631372732.1631372732,146576340.1631372727,2021-09-11,18:00:00,1,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,mobile,Android,Xiaomi,393x851,Android Webview,Russia,Saint Petersburg,0
10468046,6298572358715201935.1632655759.1632655759,1466500656.1632655759,2021-09-26,14:00:00,1,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,mobile,Android,Xiaomi,393x851,Chrome,Russia,Saint Petersburg,0
10447475,6286287940505890922.1638723693.1638723693,1463640467.1638723690,2021-12-05,20:01:33,1,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,mobile,Android,Samsung,360x800,Chrome,Russia,Saint Petersburg,0
10458104,6293039173822527781.1632581928.1632581928,1465212361.1632581925,2021-09-25,17:00:00,1,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,mobile,Android,Samsung,412x915,Chrome,Russia,Moscow,0
10462803,6295616553621347225.1621842841.1621842841,1465812454.1621842841,2021-05-24,10:00:00,1,aXQzDWsJuGXeBXexNHjc,referral,LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,desktop,Windows,custom,1920x1200,Chrome,Russia,Moscow,0


### 2.2.3 Getting jsons for sending to the model

In [81]:
# import random

# def get_jsons_from_df(n, df):
#     for _ in range(n):
#         row_number = random.randint(1, 1000000)
#         row = df.loc[row_number]
#         row_df = row.to_frame().T
#         row_df.to_json('requests/row_' + str(row_number) + '.json', orient='records', lines=True)

# quantity = 10
# get_jsons_from_df(quantity, df_sessions)

# 3. Feature Engineering

In [86]:
df_feat_eng = df_data_prep.copy()

## 3.1 Dates

In [87]:
df_feat_eng.visit_date = pd.to_datetime(df_feat_eng.visit_date)

In [88]:
df_feat_eng["visit_month"] = df_feat_eng.visit_date.dt.month
df_feat_eng["visit_day"] = df_feat_eng.visit_date.dt.day
df_feat_eng["visit_dayofweek"] = df_feat_eng.visit_date.dt.day_of_week
df_feat_eng["visit_year"] = df_feat_eng.visit_date.dt.year

In [89]:
# csv_filename = 'holidays_list.csv'

# with open(csv_filename, mode='w', newline='') as file:
#     writer = csv.writer(file)
#     for i in holidays_list:
#         writer.writerow(i)

In [90]:
def get_holidays_list():
    holidays_list = []

    with open('holidays_list.csv', mode='r', newline='') as file:
        reader = csv.reader(file)
        for row in reader:
            holiday_row = tuple(map(int, row))
            holidays_list.append(holiday_row)

    # Выводим полученный список кортежей
    return holidays_list

In [91]:
holidays_list = get_holidays_list()
df_feat_eng["day_category"] = df_feat_eng.apply(
    lambda x: 'holiday' if (x.visit_month, x.visit_day) in holidays_list
    else ('workday' if x.visit_dayofweek < 6 else 'weekend'
          ), axis=1
)

In [92]:
df_feat_eng.day_category.value_counts()

day_category
workday    1511487
weekend     210556
holiday       9914
Name: count, dtype: int64

In [93]:
df_feat_eng["season"] = df_feat_eng.visit_month.apply(
    lambda x: 'winter' if x == 12 or x == 1 or x == 2 
    else ('spring' if x < 6 else ('summer' if x < 9 else 'autumn'))
    )

df_feat_eng.season.value_counts()

season
autumn    771004
summer    490333
winter    366029
spring    104591
Name: count, dtype: int64

## 3.2 Time

Converting time into 4 categories: morning, morning_rush_hour, workhour, evening_rush_hour and evening.

In [94]:
df_feat_eng["visit_time_category"] = df_feat_eng.visit_time.apply(
    lambda x: 'night' if int(x[:2]) < 5 
        else ('morning' if int(x[:2]) < 8 
              else ('morning_rush' if int(x[:2]) < 10 
                else ('workhour' if int(x[:2]) < 17 
                    else ('evening_rush' if int(x[:2]) < 19
                        else 'evening'
                        )
                    )
                )
            )
        )
            


In [95]:
df_feat_eng.visit_time_category.value_counts()

visit_time_category
workhour        676767
evening         441043
evening_rush    188924
night           163969
morning_rush    152291
morning         108963
Name: count, dtype: int64

In [96]:
df_feat_eng.head()

,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,device_category,...,geo_country,geo_city,event_action,visit_month,visit_day,visit_dayofweek,visit_year,day_category,season,visit_time_category
12337081,7284725054902084534.1624538941.1624538941,1696107223.1607705526,2021-06-24,15:00:00,75,jaSOmLICuBzCFqHfBdRg,email,LliRUcMuIXWdLyWHGyiO,sDWYAbLNiGZVxGBDdTxc,desktop,...,Russia,Moscow,1,6,24,3,2021,workday,summer,workhour
3288869,2502982653972912560.1628266799.1628266799,582771062.1627724208,2021-08-06,19:00:00,6,kjsLglQLzykiRbcDiGcD,organic,LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,desktop,...,Russia,Moscow,1,8,6,4,2021,workday,summer,evening
10566794,63501960694916780.1640162989.1640162989,14785202.1640162988,2021-12-22,11:49:49,1,ZpYIoDJMcFzVoPFsHGJL,push,sbJRYgVfvcnqKJNDDYIr,JNHcPlZPxEMWDnRiyoBf,mobile,...,Russia,Moscow,1,12,22,2,2021,workday,winter,workhour
14538018,8447975741139857132.1640115950.1640115969,1966947629.1640115948,2021-12-21,22:46:09,1,bByPQxmDaMXgpHeypKSM,referral,QdLfySaGXolfTBSNVfHn,SOkCdPxfUcZUzzOdgGES,desktop,...,Russia,Kazan,1,12,21,1,2021,workday,winter,evening
12116165,7169040235355732641.1625062049.1625062049,1669172252.1625062049,2021-06-30,17:00:00,1,ZpYIoDJMcFzVoPFsHGJL,banner,eimRuUrNhZLAYcwRrNXu,JNHcPlZPxEMWDnRiyoBf,mobile,...,Russia,Saint Petersburg,1,6,30,2,2021,workday,summer,evening_rush


## 3.3 Screen Resolution

In [97]:
df_feat_eng.device_screen_resolution.nunique()

4947

In [98]:
df_feat_eng['device_screen_area'] = df_feat_eng.device_screen_resolution.apply(
    lambda x: int(x.split("x")[0]) * int(x.split("x")[1])
)
df_feat_eng.head()

,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,device_category,...,geo_city,event_action,visit_month,visit_day,visit_dayofweek,visit_year,day_category,season,visit_time_category,device_screen_area
12337081,7284725054902084534.1624538941.1624538941,1696107223.1607705526,2021-06-24,15:00:00,75,jaSOmLICuBzCFqHfBdRg,email,LliRUcMuIXWdLyWHGyiO,sDWYAbLNiGZVxGBDdTxc,desktop,...,Moscow,1,6,24,3,2021,workday,summer,workhour,921600
3288869,2502982653972912560.1628266799.1628266799,582771062.1627724208,2021-08-06,19:00:00,6,kjsLglQLzykiRbcDiGcD,organic,LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,desktop,...,Moscow,1,8,6,4,2021,workday,summer,evening,1327104
10566794,63501960694916780.1640162989.1640162989,14785202.1640162988,2021-12-22,11:49:49,1,ZpYIoDJMcFzVoPFsHGJL,push,sbJRYgVfvcnqKJNDDYIr,JNHcPlZPxEMWDnRiyoBf,mobile,...,Moscow,1,12,22,2,2021,workday,winter,workhour,376980
14538018,8447975741139857132.1640115950.1640115969,1966947629.1640115948,2021-12-21,22:46:09,1,bByPQxmDaMXgpHeypKSM,referral,QdLfySaGXolfTBSNVfHn,SOkCdPxfUcZUzzOdgGES,desktop,...,Kazan,1,12,21,1,2021,workday,winter,evening,694375
12116165,7169040235355732641.1625062049.1625062049,1669172252.1625062049,2021-06-30,17:00:00,1,ZpYIoDJMcFzVoPFsHGJL,banner,eimRuUrNhZLAYcwRrNXu,JNHcPlZPxEMWDnRiyoBf,mobile,...,Saint Petersburg,1,6,30,2,2021,workday,summer,evening_rush,242724


In [99]:
def categorize_device(row):
    category = row['device_category']
    area = row['device_screen_area']
    
    if category == 'mobile':
        if area < 130000:
            return 'small_mobile'
        elif area < 200000:
            return 'medium_mobile'
        else:
            return 'big_mobile'
    elif category == 'desktop':
        if area < 500000:
            return 'small_desktop'
        elif area < 1000000:
            return 'medium_desktop'
        else:
            return 'big_desktop'
    elif category == 'tablet':
        if area < 300000:
            return 'small_tablet'
        elif area < 600000:
            return 'medium_tablet'
        else:
            return 'big_tablet'
    else:
        return 'unknown'

df_feat_eng['screen_category'] = df_feat_eng.apply(categorize_device, axis=1)

df_feat_eng.screen_category.value_counts()

screen_category
big_mobile        1338118
big_desktop        305309
medium_desktop      35156
medium_mobile       30525
big_tablet          10579
small_desktop        6124
medium_tablet        4804
small_tablet         1313
small_mobile           29
Name: count, dtype: int64

## 3.4 Cities

### 3.4.1 Preparation

In [100]:
all_cities = df_feat_eng.geo_city.unique().tolist()
len(all_cities)

2386

In [101]:
# csv_filename = 'lat_long.csv'
# coordinates = {}

# with open(csv_filename, mode='r') as file:
#     reader = csv.reader(file)
#     # Пропускаем заголовок
#     next(reader)
#     # Наполнение словаря
#     for row in reader:
#         key = row[0]
#         value = eval(row[1])
#         coordinates[key] = value

# len(coordinates)

In [102]:
# coordinates

In [103]:
# for city in coordinates.keys():
#     city.replace("'", "")

# for _ in range(5):
#     if "'" in coordinates.keys():
#         print(city)

In [104]:
# cities_in_coordinates = coordinates.keys()
# cities_in_coordinates

In [105]:
# difference = all_cities - cities_in_coordinates
# difference

In [106]:
# geolocator = Nominatim(user_agent="geoapiExercises")
# geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
# location = geolocator.geocode('Moscow')
# coordinates['Moscow'] = location.latitude, location.longitude


In [107]:
# Nominatim initialization
# geolocator = Nominatim(user_agent="geoapiExercises")
# geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
# import time

# for city_name in difference:
#     time.sleep(1)
#     try:
#         location = geolocator.geocode(city_name)
#         coordinates[city_name] = location.latitude, location.longitude
#     except Exception as error:
#         print(f"Ошибка при обработке города {city_name}: {error}")


In [108]:
# geolocator = Nominatim(user_agent="geoapiExercises")
# lat, long = 55.7419774, 52.399207
# location = geolocator.reverse((lat, long), exactly_one=True)
# address = location.raw.get('address', {})
# address

In [109]:
# len(coordinates)

In [110]:
# Nominatim initialization
# geolocator = Nominatim(user_agent="geoapiExercises")
# geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
# import time

# # coordinates = dict()

# # Coordinates extraction function
# def get_coordinates(city_name):
#     if city_name in coordinates.keys():
#         return coordinates[city_name][0], coordinates[city_name][1]
#     else:
#         time.sleep(1)
#         try:
#             location = geolocator.geocode(city_name)
#             if location:
#                 coordinates[city_name] = location.latitude, location.longitude
#                 return (location.latitude, location.longitude)
#             else:
#                 return (None, None)
#         except Exception as error:
#             print(f"Ошибка при обработке города {city_name}: {error}")
#             return (None, None)
        
    
        
# # Applying the function to DataFrame to create new columns
# df_feat_eng['coordinates'] = df_feat_eng['geo_city'].apply(get_coordinates)
# df_feat_eng['latitude'] = df_feat_eng['coordinates'].apply(lambda x: x[0])
# df_feat_eng['longitude'] = df_feat_eng['coordinates'].apply(lambda x: x[1])
# df_feat_eng.drop(columns=['coordinates'], inplace=True)

In [111]:
# csv_filename = 'lat_long.csv'

# with open(csv_filename, mode='w', newline='') as file:
#     writer = csv.writer(file)
#     for key, value in coordinates.items():
#         writer.writerow([key, value])


In [112]:
# len(coordinates)

In [113]:
# city_name = "Setagaya"
# url = f"http://api.geonames.org/searchJSON?q={city_name}&maxRows=1&username={username}"
# response = requests.get(url)
# data = response.json()
# fcode_name = data['geonames'][0].get('fcodeName')
# fcode_name

In [114]:

# def get_fcodes(city, username):
#     # Формирование URL для запроса к Geonames
#     url = f"http://api.geonames.org/searchJSON?q={city}&maxRows=1&username={username}"
#     response = requests.get(url)
#     data = response.json()
    # try:
    #     if data['totalResultsCount'] > 0:
    #         fcode_name = data['geonames'][0].get('fcodeName')
    #     else:
    #         return 'Not found'
    # except Exception as e:
    #     # print(f"Error retrieving data for city of {city}")
    #     return 'Error'
    
#     return fcode_name

# city_type = []
# # Пример использования функции
# username = 'azhuravlev'  # Замените на ваше имя пользователя для доступа к API
# count = 0
# for city_name in coordinates.keys():
#     count += 1
#     fcode_name = get_fcodes(city_name, username)
#     city_type.append((city_name, fcode_name))
#     time.sleep(1)  # Задержка между запросами (1 секунда)
#     print(f"{count:<10}{city_name:<25}:\t{fcode_name}")


In [115]:
# city_type

In [116]:
# not_found_list = [item[0] for item in city_type_all if item[1] == 'Not found']
# city_type_to_csv = [item for item in city_type if item[1] != 'Not found']

# len(city_type_to_csv), len(not_found_list)

In [117]:
# city_type_not_found = []
# username = 'azhuravlev'  # Замените на ваше имя пользователя для доступа к API
# count = 0
# for city_name in not_found_list:
#     count += 1
#     fcode_name = get_fcodes(city_name, username)
#     city_type_not_found.append((city_name, fcode_name))
#     time.sleep(1)  # Задержка между запросами (1 секунда)
#     print(f"{count:<10}{city_name:<25}:\t{fcode_name}")


In [118]:
# city_type_all = city_type_to_csv + city_type_not_found
# len(city_type_all)

In [119]:
# csv_filename = 'city_type.csv'

# with open(csv_filename, mode='w', newline='') as file:
#     writer = csv.writer(file)
#     for i in city_type_all:
#         writer.writerow(i)

In [120]:
# places = set([x[1] for x in city_type_all])
# places

### 3.4.2 Working with administrative divisions


In [121]:
csv_filename = 'updated_city_type.csv'
city_type_dict = {}

with open(csv_filename, mode='r') as file:
    reader = csv.reader(file)
    for row in reader:
        key = row[0]
        value = row[1]
        city_type_dict[key] = value
    
print(len(city_type_dict))

2386


In [122]:
def get_city_type_from_geonames(city):
    username = 'azhuravlev'
    url = f"http://api.geonames.org/searchJSON?q={city}&maxRows=1&username={username}"
    response = requests.get(url)
    data = response.json()
    try:
        if data['totalResultsCount'] > 0:
            description = data['geonames'][0].get('fcodeName').split()
            if 'first-order' in description:
                return 'first'
            elif 'second-order' in description:
                return 'second'
            elif 'third-order' in description:
                return 'third'
            elif 'capital' in description:
                return 'capital'
            elif 'abandoned' in description:
                return 'abandoned'
            else:
                return 'place'
        else:
            return 'abandoned place'
    except Exception:
        return 'abandoned place'

def determine_city_type(city_name, city_type_dict):
    if city_name in city_type_dict.keys():
        city_type = city_type_dict[city_name]
    else:
        city_type = get_city_type_from_geonames(city_name)
    return city_type

df_feat_eng['city_type'] = df_feat_eng.geo_city.apply(lambda city: determine_city_type(city, city_type_dict))



In [123]:
df_feat_eng[['geo_city', 'city_type']].head(10)

,geo_city,city_type
12337081,Moscow,capital
3288869,Moscow,capital
10566794,Moscow,capital
14538018,Kazan,first
12116165,Saint Petersburg,first
10490314,Moscow,capital
4773100,Grozny,first
10603173,(not set),abandoned
4198915,Vladivostok,first
7072397,Moscow,capital


## 3.8 Country

In [124]:
former_ussr_countries = [
    "Armenia",
    "Azerbaijan",
    "Belarus",
    "Estonia",
    "Georgia",
    "Kazakhstan",
    "Kyrgyzstan",
    "Latvia",
    "Lithuania",
    "Moldova",
    "Tajikistan",
    "Turkmenistan",
    "Ukraine",
    "Uzbekistan",
    "Russia"
]
former_ussr_countries

['Armenia',
 'Azerbaijan',
 'Belarus',
 'Estonia',
 'Georgia',
 'Kazakhstan',
 'Kyrgyzstan',
 'Latvia',
 'Lithuania',
 'Moldova',
 'Tajikistan',
 'Turkmenistan',
 'Ukraine',
 'Uzbekistan',
 'Russia']

In [125]:
df_feat_eng['USSR'] = df_feat_eng.geo_country.apply(
    lambda x: 1 if x in former_ussr_countries else 0
)

df_feat_eng['USSR'].head()

12337081    1
3288869     1
10566794    1
14538018    1
12116165    1
Name: USSR, dtype: int64

In [126]:
df_feat_eng.head()

,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,device_category,...,visit_day,visit_dayofweek,visit_year,day_category,season,visit_time_category,device_screen_area,screen_category,city_type,USSR
12337081,7284725054902084534.1624538941.1624538941,1696107223.1607705526,2021-06-24,15:00:00,75,jaSOmLICuBzCFqHfBdRg,email,LliRUcMuIXWdLyWHGyiO,sDWYAbLNiGZVxGBDdTxc,desktop,...,24,3,2021,workday,summer,workhour,921600,medium_desktop,capital,1
3288869,2502982653972912560.1628266799.1628266799,582771062.1627724208,2021-08-06,19:00:00,6,kjsLglQLzykiRbcDiGcD,organic,LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,desktop,...,6,4,2021,workday,summer,evening,1327104,big_desktop,capital,1
10566794,63501960694916780.1640162989.1640162989,14785202.1640162988,2021-12-22,11:49:49,1,ZpYIoDJMcFzVoPFsHGJL,push,sbJRYgVfvcnqKJNDDYIr,JNHcPlZPxEMWDnRiyoBf,mobile,...,22,2,2021,workday,winter,workhour,376980,big_mobile,capital,1
14538018,8447975741139857132.1640115950.1640115969,1966947629.1640115948,2021-12-21,22:46:09,1,bByPQxmDaMXgpHeypKSM,referral,QdLfySaGXolfTBSNVfHn,SOkCdPxfUcZUzzOdgGES,desktop,...,21,1,2021,workday,winter,evening,694375,medium_desktop,first,1
12116165,7169040235355732641.1625062049.1625062049,1669172252.1625062049,2021-06-30,17:00:00,1,ZpYIoDJMcFzVoPFsHGJL,banner,eimRuUrNhZLAYcwRrNXu,JNHcPlZPxEMWDnRiyoBf,mobile,...,30,2,2021,workday,summer,evening_rush,242724,big_mobile,first,1


## 3.9 Device brand

In [127]:
df_feat_eng.device_brand.value_counts()

device_brand
Apple        526948
custom       323060
Samsung      311641
Xiaomi       269251
Huawei       173828
              ...  
Gigaset           1
M-HORSE           1
Tonbux            1
Cube              1
Honeywell         1
Name: count, Length: 202, dtype: int64

In [128]:
dev_brand_count = df_feat_eng.device_brand.value_counts().to_dict()
df_feat_eng["device_brand_cat"] = df_feat_eng["device_brand"].apply(
    lambda x: x if dev_brand_count[x] > 100000 else 'other'
    )


df_feat_eng.device_brand_cat.value_counts()

device_brand_cat
Apple      526948
custom     323060
Samsung    311641
Xiaomi     269251
Huawei     173828
other      127229
Name: count, dtype: int64

## 3.10 Device OS

In [129]:
df_feat_eng.device_os.value_counts()

device_os
Android          875038
iOS              503538
Windows          318491
Macintosh         23415
other              6754
Linux              4616
Chrome OS            65
BlackBerry           24
Tizen                 7
Firefox OS            3
Windows Phone         2
Nokia                 2
Samsung               2
Name: count, dtype: int64

In [130]:
dev_os_count = df_feat_eng.device_os.value_counts().to_dict()
df_feat_eng['device_os_cat'] = df_feat_eng.device_os.apply(
    lambda x: x if dev_os_count[x] > 1000 else 'other'
)
df_feat_eng.device_os_cat.value_counts()

device_os_cat
Android      875038
iOS          503538
Windows      318491
Macintosh     23415
other          6859
Linux          4616
Name: count, dtype: int64

## 3.11 Device browser

In [131]:
df_feat_eng.device_browser.value_counts()

device_browser
Chrome                              951561
Safari                              436705
YaBrowser                           123765
Safari (in-app)                      65952
Android Webview                      51919
Samsung Internet                     44273
Opera                                25094
Firefox                              15738
Edge                                 15196
UC Browser                             815
Android Runtime                        241
[FBAN                                  109
Mozilla Compatible Agent                80
MRCHROME                                72
Instagram 208.0.0.32.135 Android        72
Instagram 209.0.0.21.119 Android        49
Android Browser                         47
Puffin                                  43
Maxthon                                 30
Internet Explorer                       24
Instagram 202.0.0.23.119                17
Android                                 16
Mozilla                                

In [132]:
dev_browser = df_feat_eng.device_browser.value_counts().to_dict()
df_feat_eng["device_browser_cat"] = df_feat_eng["device_browser"].apply(
    lambda x: x if dev_browser[x] > 100000 else 'other'
    )


df_feat_eng.device_browser_cat.value_counts()

device_browser_cat
Chrome       951561
Safari       436705
other        219926
YaBrowser    123765
Name: count, dtype: int64

# 4.Conversion & Standardization

## 4.1 Categorical

In [133]:
df_conversion = df_feat_eng.copy()

In [134]:
categories = df_conversion.columns

for i in categories:
    print(f"{i:>25}: {df_conversion[i].nunique()}")

               session_id: 1731957
                client_id: 1320396
               visit_date: 226
               visit_time: 85028
             visit_number: 533
               utm_source: 280
               utm_medium: 55
             utm_campaign: 406
            utm_adcontent: 280
          device_category: 3
                device_os: 13
             device_brand: 202
 device_screen_resolution: 4947
           device_browser: 51
              geo_country: 159
                 geo_city: 2386
             event_action: 2
              visit_month: 8
                visit_day: 31
          visit_dayofweek: 7
               visit_year: 1
             day_category: 3
                   season: 4
      visit_time_category: 6
       device_screen_area: 4506
          screen_category: 9
                city_type: 6
                     USSR: 2
         device_brand_cat: 6
            device_os_cat: 6
       device_browser_cat: 4


In [135]:
categories_to_ohe = [
    "visit_time_category",
    "device_category",
    "device_brand_cat",
    "day_category",
    "season",
    "screen_category",
    "device_os_cat",
    "device_browser_cat",
    "city_type",
    
    ]


In [136]:
for i in categories_to_ohe:
    print(f"{i:>25}: {df_conversion[i].nunique()}")

      visit_time_category: 6
          device_category: 3
         device_brand_cat: 6
             day_category: 3
                   season: 4
          screen_category: 9
            device_os_cat: 6
       device_browser_cat: 4
                city_type: 6


Creating a new df for encoding:

In [137]:
data = df_conversion[categories_to_ohe]
data.head()

,visit_time_category,device_category,device_brand_cat,day_category,season,screen_category,device_os_cat,device_browser_cat,city_type
12337081,workhour,desktop,custom,workday,summer,medium_desktop,Windows,Chrome,capital
3288869,evening,desktop,custom,workday,summer,big_desktop,Windows,other,capital
10566794,workhour,mobile,Samsung,workday,winter,big_mobile,Android,Chrome,capital
14538018,evening,desktop,custom,workday,winter,medium_desktop,Windows,Chrome,first
12116165,evening_rush,mobile,Samsung,workday,summer,big_mobile,Android,Chrome,first


In [138]:
ohe = OneHotEncoder(sparse_output=False)

In [139]:
ohe_data = ohe.fit_transform(data)
print(f"{ohe_data.shape}\n\n{ohe.get_feature_names_out()}")

(1731957, 47)

['visit_time_category_evening' 'visit_time_category_evening_rush'
 'visit_time_category_morning' 'visit_time_category_morning_rush'
 'visit_time_category_night' 'visit_time_category_workhour'
 'device_category_desktop' 'device_category_mobile'
 'device_category_tablet' 'device_brand_cat_Apple'
 'device_brand_cat_Huawei' 'device_brand_cat_Samsung'
 'device_brand_cat_Xiaomi' 'device_brand_cat_custom'
 'device_brand_cat_other' 'day_category_holiday' 'day_category_weekend'
 'day_category_workday' 'season_autumn' 'season_spring' 'season_summer'
 'season_winter' 'screen_category_big_desktop'
 'screen_category_big_mobile' 'screen_category_big_tablet'
 'screen_category_medium_desktop' 'screen_category_medium_mobile'
 'screen_category_medium_tablet' 'screen_category_small_desktop'
 'screen_category_small_mobile' 'screen_category_small_tablet'
 'device_os_cat_Android' 'device_os_cat_Linux' 'device_os_cat_Macintosh'
 'device_os_cat_Windows' 'device_os_cat_iOS' 'device_os_cat_other'

## 4.2 Standardization

In [140]:
categories_to_std_scaler = ['visit_number']

In [141]:
data_std = df_conversion[categories_to_std_scaler]
data_std.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1731957 entries, 12337081 to 5245208
Data columns (total 1 columns):
 #   Column        Dtype
---  ------        -----
 0   visit_number  Int32
dtypes: Int32(1)
memory usage: 21.5 MB


In [142]:
std_scaler = StandardScaler()

In [143]:
std_scaler_cats_std = std_scaler.fit_transform(data_std)
print(std_scaler_cats_std.shape)

(1731957, 1)


In [144]:
features_std = [i + "_std" for i in std_scaler.get_feature_names_out()]
features_std

['visit_number_std']

In [145]:
df_conversion[features_std] = std_scaler_cats_std
df_conversion.head(3)

,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,device_category,...,season,visit_time_category,device_screen_area,screen_category,city_type,USSR,device_brand_cat,device_os_cat,device_browser_cat,visit_number_std
12337081,7284725054902084534.1624538941.1624538941,1696107223.1607705526,2021-06-24,15:00:00,75,jaSOmLICuBzCFqHfBdRg,email,LliRUcMuIXWdLyWHGyiO,sDWYAbLNiGZVxGBDdTxc,desktop,...,summer,workhour,921600,medium_desktop,capital,1,custom,Windows,Chrome,6.215585
3288869,2502982653972912560.1628266799.1628266799,582771062.1627724208,2021-08-06,19:00:00,6,kjsLglQLzykiRbcDiGcD,organic,LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,desktop,...,summer,evening,1327104,big_desktop,capital,1,custom,Windows,other,0.289590
10566794,63501960694916780.1640162989.1640162989,14785202.1640162988,2021-12-22,11:49:49,1,ZpYIoDJMcFzVoPFsHGJL,push,sbJRYgVfvcnqKJNDDYIr,JNHcPlZPxEMWDnRiyoBf,mobile,...,winter,workhour,376980,big_mobile,capital,1,Samsung,Android,Chrome,-0.139830


In [146]:
df_conversion.shape

(1731957, 32)

In [147]:
df_categorical = pd.DataFrame(ohe_data, columns=ohe.get_feature_names_out())
df_conversion = pd.concat(
    [df_conversion.reset_index(drop=True), df_categorical.reset_index(drop=True)], 
    axis=1, 
    ignore_index=False
    )

df_categorical.shape, df_conversion.shape

((1731957, 47), (1731957, 79))

In [148]:
df_conversion.columns

Index(['session_id', 'client_id', 'visit_date', 'visit_time', 'visit_number',
       'utm_source', 'utm_medium', 'utm_campaign', 'utm_adcontent',
       'device_category', 'device_os', 'device_brand',
       'device_screen_resolution', 'device_browser', 'geo_country', 'geo_city',
       'event_action', 'visit_month', 'visit_day', 'visit_dayofweek',
       'visit_year', 'day_category', 'season', 'visit_time_category',
       'device_screen_area', 'screen_category', 'city_type', 'USSR',
       'device_brand_cat', 'device_os_cat', 'device_browser_cat',
       'visit_number_std', 'visit_time_category_evening',
       'visit_time_category_evening_rush', 'visit_time_category_morning',
       'visit_time_category_morning_rush', 'visit_time_category_night',
       'visit_time_category_workhour', 'device_category_desktop',
       'device_category_mobile', 'device_category_tablet',
       'device_brand_cat_Apple', 'device_brand_cat_Huawei',
       'device_brand_cat_Samsung', 'device_brand_cat_Xi

Dropping unnecessary features:

In [149]:
str_features = [i for i in df_conversion.columns if df_conversion[i].dtype == "O"]
print(str_features)
ids = ['session_id', 'client_id', 'visit_date', 'visit_year', 'geo_country', 'geo_city']
cats_to_drop = ids + categories_to_std_scaler + categories_to_ohe + str_features
# cats_to_drop = ids + categories_to_std_scaler + categories_to_ohe
# cats_to_drop = ids + categories_to_std_scaler
df_conversion = df_conversion.drop(df_conversion[cats_to_drop], axis=1)
df_conversion.shape

['session_id', 'client_id', 'visit_time', 'utm_source', 'utm_medium', 'utm_campaign', 'utm_adcontent', 'device_category', 'device_os', 'device_brand', 'device_screen_resolution', 'device_browser', 'geo_country', 'geo_city', 'day_category', 'season', 'visit_time_category', 'screen_category', 'city_type', 'device_brand_cat', 'device_os_cat', 'device_browser_cat']


(1731957, 54)

In [150]:
df_conversion.columns

Index(['event_action', 'visit_month', 'visit_day', 'visit_dayofweek',
       'device_screen_area', 'USSR', 'visit_number_std',
       'visit_time_category_evening', 'visit_time_category_evening_rush',
       'visit_time_category_morning', 'visit_time_category_morning_rush',
       'visit_time_category_night', 'visit_time_category_workhour',
       'device_category_desktop', 'device_category_mobile',
       'device_category_tablet', 'device_brand_cat_Apple',
       'device_brand_cat_Huawei', 'device_brand_cat_Samsung',
       'device_brand_cat_Xiaomi', 'device_brand_cat_custom',
       'device_brand_cat_other', 'day_category_holiday',
       'day_category_weekend', 'day_category_workday', 'season_autumn',
       'season_spring', 'season_summer', 'season_winter',
       'screen_category_big_desktop', 'screen_category_big_mobile',
       'screen_category_big_tablet', 'screen_category_medium_desktop',
       'screen_category_medium_mobile', 'screen_category_medium_tablet',
       'screen_c

In [151]:
count = 0
for i in df_conversion.columns:
    if df_conversion[i].dtype == "O":
        count += 1

if count == 0:
    print("No string features found.")

No string features found.


In [152]:
# df_conversion.drop(["event_action"], axis=1).corrwith(df_feat_eng.event_action).sort_values(ascending=False)

# 5. Modelling

In [153]:
df_model = df_conversion.copy()

In [154]:
cols = df_model.columns.to_list()
cols

['event_action',
 'visit_month',
 'visit_day',
 'visit_dayofweek',
 'device_screen_area',
 'USSR',
 'visit_number_std',
 'visit_time_category_evening',
 'visit_time_category_evening_rush',
 'visit_time_category_morning',
 'visit_time_category_morning_rush',
 'visit_time_category_night',
 'visit_time_category_workhour',
 'device_category_desktop',
 'device_category_mobile',
 'device_category_tablet',
 'device_brand_cat_Apple',
 'device_brand_cat_Huawei',
 'device_brand_cat_Samsung',
 'device_brand_cat_Xiaomi',
 'device_brand_cat_custom',
 'device_brand_cat_other',
 'day_category_holiday',
 'day_category_weekend',
 'day_category_workday',
 'season_autumn',
 'season_spring',
 'season_summer',
 'season_winter',
 'screen_category_big_desktop',
 'screen_category_big_mobile',
 'screen_category_big_tablet',
 'screen_category_medium_desktop',
 'screen_category_medium_mobile',
 'screen_category_medium_tablet',
 'screen_category_small_desktop',
 'screen_category_small_mobile',
 'screen_category_s

## 5.1 Initialization and dataset splitting

In [155]:
X = df_model.drop(columns=["event_action"])
y = df_model.event_action

In [156]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

## 5.2 Models initialization with default parameters

### 5.2.1 XGBClaccifier

In [157]:
xgb = XGBClassifier(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=8,
    min_child_weight=5,
    subsample=0.8,
    colsample_bytree=0.8,
    objective='binary:logistic',
    reg_lambda=1,
    gamma=1,
    alpha=0.1,
    tree_method='hist',
    eval_metric='auc',
)

# 66,0177:
#     n_estimators=300,
#     learning_rate=0.05,
#     max_depth=8,
#     min_child_weight=5,
#     subsample=0.8,
#     colsample_bytree=0.8,
#     objective='binary:logistic',
#     gamma=1,
#     alpha=0.1,
#     tree_method='hist',
#     eval_metric='auc',


xgb.fit(x_train, y_train)

XGBClassifier(alpha=0.1, base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='auc', feature_types=None,
              gamma=1, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=8, max_leaves=None,
              min_child_weight=5, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=300, n_jobs=None,
              num_parallel_tree=None, ...)

In [158]:
roc_auc_score(y_test, xgb.predict_proba(x_test)[:, 1])

0.6601774308518131

In [159]:
# cv_score_xgb = cross_val_score(xgb, X, y, cv=5, n_jobs=-1, scoring='roc_auc')
# print(cv_score_xgb.mean(), cv_score_xgb.std())

### 5.2.2 LightGBM

In [160]:
# lgb = LGBMClassifier(
#     n_estimators=1000,
#     learning_rate=0.05,
#     max_depth=8,
#     num_leaves=31,
#     min_data_in_leaf=20,
#     feature_fraction=0.8,
#     bagging_fraction=0.7,
#     bagging_freq=5,
#     objective='binary',
# )


# lgb.fit(x_train, y_train)

In [161]:
# roc_auc_score(y_test, lgb.predict_proba(x_test)[:, 1])

In [162]:
# cv_score_lgb = cross_val_score(lgb, X, Y, cv=5, n_jobs=-1, scoring='roc_auc')
# print(cv_score_lgb.mean(), cv_score_lgb.std())

### 5.2.3 CatBoost

In [163]:
# cgb = CatBoostClassifier(
#     iterations=1000,
#     learning_rate=0.05,
#     depth=8,
#     l2_leaf_reg=5,
#     random_strength=2,
#     bagging_temperature=0.8,
#     loss_function='Logloss',
#     eval_metric='AUC'
# )


# cgb.fit(x_train, y_train)

In [164]:
# roc_auc_score(y_test, cgb.predict_proba(x_test)[:, 1])

In [165]:
# cv_score_cgb = cross_val_score(cgb, X, Y, cv=5, n_jobs=-1, scoring='roc_auc')
# print(cv_score_cgb.mean(), cv_score_cgb.std())

### 5.2.4 GradientBoosting

In [166]:
# gb_upd = GradientBoostingClassifier(
#     n_estimators=185,
#     learning_rate=0.34,
#     max_depth=5,
#     min_samples_split=3,
#     min_samples_leaf=2,
#     subsample=.8,
#     max_features='sqrt',
#     random_state=42
#     # **best_params, random_state=42
#     )
# gb_upd.fit(x_train, y_train)

# Параметры для 65,334
    # n_estimators=185,
    # learning_rate=0.34,
    # max_depth=5,
    # min_samples_split=3,
    # min_samples_leaf=2,
    # subsample=0.8,
    # max_features='sqrt',
    # random_state=42


In [167]:
# roc_auc_score(y_test, gb_upd.predict_proba(x_test)[:, 1])

In [168]:
# cv_score_gb = cross_val_score(gb_upd, X, Y, cv=5, n_jobs=-1, scoring='roc_auc')
# print(cv_score_gb.mean(), cv_score_gb.std())


In [169]:
# cv_score_gb

### 5.3 Default models' accuracy score

In [170]:
# roc_auc_score(y_test, logreg_upd.predict_proba(X)[:, 1])

In [171]:
# roc_auc_score(y_test, rf_upd.predict_proba(x_test)[:, 1])

In [172]:
# roc_auc_score(y_test, gb_upd.predict_proba(x_test)[:, 1])

In [173]:
# cv_score_gb = cross_val_score(gb_upd, X, Y, cv=5, n_jobs=-1, scoring='roc_auc')
# print(cv_score_gb.mean(), cv_score_gb.std())


In [174]:
# cv = cv_score_rf.tolist()
# cv

In [175]:
# cross_validate(rf_upd, X, Y, cv=5, n_jobs=-1, scoring='roc_auc')

| version | value |
|----------|----------|
| april | 0.6294236932989987 |
| date&time | 0.6385665035825867 |
| date&time corrected season | 0.6368971213627024 |
| screen res, dev brand, dev os | 0.6397631389436769 |
| dev_br_cat, dev_os_cat, geo_rus | 0.6374147085777188 |
| -geo_rus, dev_browser_cat | 0.6432028498326705 |
| don't remember | 0.6424442050896509 |
| USSR | 0.6429456076315219 |
| city_type w/ error | 0.6431313276906813 |
| city_type & upgraded GBC | 0.6474265163352543 |
| city_type by 0,1,2 | 0.6469685224145244 |
| city_type + 'abandoned'| 0.648259935681126 |
| upgraded GBC | 0.6520331385846819 |




